# 🍎 Apple Disease Classification - CNN-BiLSTM with Comprehensive Metrics

## Enhanced Performance Evaluation Including:
- **Bidirectional LSTM** for better spatial feature learning
- **ROC-AUC curves** (per-class and micro/macro averages)
- **Precision-Recall curves** with average precision scores
- **Confusion matrices** (raw counts and normalized)
- **Per-class detailed metrics** (Precision, Recall, F1, Support)
- **Matthews Correlation Coefficient** (MCC)
- **Sample inference visualization** with confidence scores
- **Comprehensive metrics export** to JSON and CSV

## 1️⃣ Mount Drive and Setup Paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, glob

# Auto-resolve AppleWork base directory
candidates = [
    "/content/drive/MyDrive/Abebe_Colab/AppleWork",
    "/content/drive/My Drive/Abebe_Colab/AppleWork",
    "/content/Drive/MyDrive/Abebe_Colab/AppleWork",
    "/content/Drive/My Drive/Abebe_Colab/AppleWork",
]

BASE_DIR = next((p for p in candidates if os.path.exists(p)), None)
if BASE_DIR is None:
    hits = glob.glob("/content/**/AppleWork", recursive=True)
    BASE_DIR = hits[0] if hits else None

assert BASE_DIR is not None, "❌ Couldn't find AppleWork directory"

DATA_DIR = os.path.join(BASE_DIR, "Apple_disease")
train_candidates = [os.path.join(DATA_DIR, n) for n in ("Train", "train")]
IMAGE_ROOT = next((p for p in train_candidates if os.path.exists(p)), None)
assert IMAGE_ROOT is not None, f"❌ Couldn't find Train/ under {DATA_DIR}"

ANNOT_PATH = os.path.join(DATA_DIR, "annotation", "train.json")

print("✅ BASE_DIR:", BASE_DIR)
print("✅ IMAGE_ROOT:", IMAGE_ROOT)
print("✅ Annotation exists:", os.path.exists(ANNOT_PATH))
print("✅ Classes:", os.listdir(IMAGE_ROOT))

Mounted at /content/drive
✅ BASE_DIR: /content/drive/MyDrive/Abebe_Colab/AppleWork
✅ IMAGE_ROOT: /content/drive/MyDrive/Abebe_Colab/AppleWork/Apple_disease/Train
✅ Annotation exists: True
✅ Classes: ['Healthy', 'Alternaria', 'Apple_Rust', 'Apple_Scab']


## 2️⃣ Import Libraries and Setup Configuration

In [ ]:
import json
import random
import itertools
import math
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score,
    auc,
    f1_score,
    accuracy_score,
    matthews_corrcoef
)
from sklearn.preprocessing import label_binarize

# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
VAL_SPLIT = 0.15
SEED = 42
LR = 1e-4

MODEL_DIR = os.path.join(BASE_DIR, "apple_cnn_bilstm_artifacts")
os.makedirs(MODEL_DIR, exist_ok=True)

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
AUTOTUNE = tf.data.AUTOTUNE

print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU available:", tf.config.list_physical_devices('GPU'))
print("✅ Model directory:", MODEL_DIR)

✅ TensorFlow version: 2.19.0
✅ GPU available: []
✅ Model directory: /content/drive/MyDrive/Abebe_Colab/AppleWork/apple_cnn_bilstm_artifacts


## 3️⃣ Dataset Preparation

In [ ]:
def list_images_by_class(root_dir):
    """List all images organized by class."""
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}
    class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    class_names = sorted(class_names)

    recs = []
    for cname in class_names:
        cdir = os.path.join(root_dir, cname)
        for fn in os.listdir(cdir):
            if os.path.splitext(fn.lower())[1] in exts:
                recs.append({"path": os.path.join(cdir, fn), "label_name": cname})

    recs.sort(key=lambda r: r["path"])
    return recs, class_names

# Load dataset
records, class_names = list_images_by_class(IMAGE_ROOT)
assert records, f"❌ No images found in {IMAGE_ROOT}"

# Create label mappings
idx_to_name = {i: n for i, n in enumerate(class_names)}
name_to_idx = {n: i for i, n in idx_to_name.items()}
num_classes = len(class_names)

print(f"✅ Total Images: {len(records)}")
print(f"✅ Classes ({num_classes}): {class_names}")

# Show class distribution
class_counts = Counter([r["label_name"] for r in records])
print("\n📊 Class Distribution:")
for cls, count in sorted(class_counts.items()):
    print(f"   {cls}: {count} images ({count/len(records)*100:.1f}%)")

✅ Total Images: 6080
✅ Classes (4): ['Alternaria', 'Apple_Rust', 'Apple_Scab', 'Healthy']

📊 Class Distribution:
   Alternaria: 1480 images (24.3%)
   Apple_Rust: 1520 images (25.0%)
   Apple_Scab: 1480 images (24.3%)
   Healthy: 1600 images (26.3%)


## 4️⃣ Train-Validation Split

In [ ]:
def stratified_split_by_name(recs, val_ratio=0.15, seed=SEED):
    """Stratified split maintaining class balance."""
    by_lbl = defaultdict(list)
    for r in recs:
        by_lbl[r["label_name"]].append(r)

    rng = random.Random(seed)
    train, val = [], []

    for lbl, items in by_lbl.items():
        rng.shuffle(items)
        n_val = max(1, int(round(val_ratio * len(items))))
        val.extend(items[:n_val])
        train.extend(items[n_val:])

    rng.shuffle(train)
    rng.shuffle(val)
    return train, val

train_recs, val_recs = stratified_split_by_name(records, VAL_SPLIT, SEED)

print(f"✅ Training samples: {len(train_recs)} ({len(train_recs)/len(records)*100:.1f}%)")
print(f"✅ Validation samples: {len(val_recs)} ({len(val_recs)/len(records)*100:.1f}%)")

✅ Training samples: 5168 (85.0%)
✅ Validation samples: 912 (15.0%)


## 5️⃣ Create TensorFlow Datasets

In [ ]:
def decode_img(path):
    """Load and decode image."""
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

@tf.function
def augment(img):
    """Data augmentation."""
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, 0.1)
    img = tf.image.random_contrast(img, 0.9, 1.1)
    return img

def build_ds(samples, training=True, batch_size=BATCH_SIZE):
    """Build TensorFlow dataset."""
    paths = [s["path"] for s in samples]
    labels = [name_to_idx[s["label_name"]] for s in samples]

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _load(p, y):
        x = decode_img(p)
        if training:
            x = augment(x)
        x = tf.keras.applications.efficientnet.preprocess_input(x * 255.0)
        return x, tf.one_hot(y, depth=num_classes)

    if training:
        ds = ds.shuffle(len(samples), seed=SEED, reshuffle_each_iteration=True)

    ds = ds.map(_load, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)

    return ds

# Create datasets
train_ds = build_ds(train_recs, training=True)
val_ds = build_ds(val_recs, training=False)

print(f"✅ Training dataset: {len(train_recs)} samples")
print(f"✅ Validation dataset: {len(val_recs)} samples")
print(f"✅ Batch size: {BATCH_SIZE}")

✅ Training dataset: 5168 samples
✅ Validation dataset: 912 samples
✅ Batch size: 32


## 6️⃣ Build CNN-BiLSTM Model

**Architecture:**
- EfficientNetB0 (pretrained on ImageNet)
- Reshape to sequence format (H×W, C)
- **Bidirectional LSTM** (256 units)
- Dense layers with dropout
- Softmax output

In [ ]:
def make_model(num_classes):
    """Build CNN-BiLSTM model."""
    inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))

    # EfficientNetB0 backbone
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs
    )

    x = base.output  # (None, H, W, C)
    h, w, c = x.shape[1], x.shape[2], x.shape[3]

    # Reshape for sequence processing
    x = tf.keras.layers.Reshape((h * w, c))(x)  # (None, T=H*W, C)

    # Bidirectional LSTM
    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(256, return_sequences=False, dropout=0.25)
    )(x)

    # Dense layers
    x = tf.keras.layers.Dropout(0.35)(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    # Output
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs, name='CNN_BiLSTM_Model')

    # Freeze early layers of EfficientNet
    for layer in base.layers[:-20]:
        layer.trainable = False

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc", multi_label=False)]
    )

    return model

# Build model
model = make_model(num_classes)
model.summary()

print("\n✅ CNN-BiLSTM model built successfully")
print(f"✅ Total parameters: {model.count_params():,}")
print(f"✅ Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "CNN_BiLSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 7,329,703 (27.96 MB)

 Trainable params: 4,631,092 (17.67 MB)

 Non-trainable params: 2,698,611 (10.29 MB)


✅ CNN-BiLSTM model built successfully
✅ Total parameters: 7,329,703
✅ Trainable parameters: 4,631,092


## 7️⃣ Setup Callbacks

In [ ]:
best_ckpt = os.path.join(MODEL_DIR, "best_cnn_bilstm.keras")

callbacks = [
    ModelCheckpoint(
        best_ckpt,
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    ),
    EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True,
        mode="max",
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Callbacks configured:")
print("   - ModelCheckpoint (save best model)")
print("   - EarlyStopping (patience=5)")
print("   - ReduceLROnPlateau (factor=0.5)")

✅ Callbacks configured:
   - ModelCheckpoint (save best model)
   - EarlyStopping (patience=5)
   - ReduceLROnPlateau (factor=0.5)


## 8️⃣ Train Model

In [ ]:
print("\n" + "="*80)
print("🚀 STARTING TRAINING")
print("="*80)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*80)
print("✅ TRAINING COMPLETED")
print("="*80)


🚀 STARTING TRAINING
Epoch 1/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5107 - auc: 0.7638 - loss: 1.1201
Epoch 1: val_accuracy improved from -inf to 0.93750, saving model to /content/drive/MyDrive/Abebe_Colab/AppleWork/apple_cnn_bilstm_artifacts/best_cnn_bilstm.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 915s 5s/step - accuracy: 0.5117 - auc: 0.7646 - loss: 1.1184 - val_accuracy: 0.9375 - val_auc: 0.9911 - val_loss: 0.2201 - learning_rate: 1.0000e-04
Epoch 2/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9192 - auc: 0.9885 - loss: 0.2614
Epoch 2: val_accuracy improved from 0.93750 to 0.97478, saving model to /content/drive/MyDrive/Abebe_Colab/AppleWork/apple_cnn_bilstm_artifacts/best_cnn_bilstm.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 868s 5s/step - accuracy: 0.9192 - auc: 0.9885 - loss: 0.2612 - val_accuracy: 0.9748 - val_auc: 0.9968 - val_loss: 0.0896 - learning_rate: 1.0000e-04
Epoch 3/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9644 - auc: 0.9959 - loss: 0.13

## 9️⃣ Plot Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('📈 Training History', fontsize=16, fontweight='bold')

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2, marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2, marker='s')
axes[0].set_xlabel('Epoch', fontweight='bold')
axes[0].set_ylabel('Accuracy', fontweight='bold')
axes[0].set_title('Model Accuracy', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2, marker='o')
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2, marker='s')
axes[1].set_xlabel('Epoch', fontweight='bold')
axes[1].set_ylabel('Loss', fontweight='bold')
axes[1].set_title('Model Loss', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# AUC
axes[2].plot(history.history['auc'], label='Train', linewidth=2, marker='o')
axes[2].plot(history.history['val_auc'], label='Validation', linewidth=2, marker='s')
axes[2].set_xlabel('Epoch', fontweight='bold')
axes[2].set_ylabel('AUC', fontweight='bold')
axes[2].set_title('Model AUC', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'training_history.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training history saved")
print(f"   Best val_accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"   Best val_auc: {max(history.history['val_auc']):.4f}")

## 🔟 Load Best Model and Generate Predictions

In [ ]:
# Load best model
model = tf.keras.models.load_model(best_ckpt)
print(f"✅ Loaded best model from: {best_ckpt}")

# Generate predictions
print("\n🔮 Generating predictions on validation set...")
y_true = []
y_pred = []
y_pred_proba = []

for images, labels in val_ds:
    preds = model.predict(images, verbose=0)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))
    y_pred_proba.extend(preds)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

print(f"✅ Predictions generated")
print(f"   Shape: {y_pred_proba.shape}")
print(f"   Samples: {len(y_true)}")

## 1️⃣1️⃣ Calculate Comprehensive Metrics

In [ ]:
# Overall metrics
overall_accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
mcc = matthews_corrcoef(y_true, y_pred)

# Classification report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Save classification report
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(os.path.join(MODEL_DIR, 'classification_report.csv'))

print("\n" + "="*80)
print("📊 OVERALL METRICS")
print("="*80)
print(f"Accuracy:              {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
print(f"Macro F1-Score:        {macro_f1:.4f}")
print(f"Weighted F1-Score:     {weighted_f1:.4f}")
print(f"Matthews Corr Coef:    {mcc:.4f}")
print("="*80)

print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

## 1️⃣2️⃣ Confusion Matrices

In [ ]:
# Calculate confusion matrices
cm = confusion_matrix(y_true, y_pred)
cm_normalized = confusion_matrix(y_true, y_pred, normalize='true')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('📊 Confusion Matrices', fontsize=16, fontweight='bold')

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0], cbar_kws={'label': 'Count'}, square=True)
axes[0].set_title('Confusion Matrix (Counts)', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Predicted Label', fontweight='bold')
axes[0].set_ylabel('True Label', fontweight='bold')

# Normalized
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[1], cbar_kws={'label': 'Proportion'}, square=True)
axes[1].set_title('Confusion Matrix (Normalized)', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Predicted Label', fontweight='bold')
axes[1].set_ylabel('True Label', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'confusion_matrices.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrices saved")

## 1️⃣3️⃣ ROC-AUC Curves

In [ ]:
# Binarize labels for ROC-AUC
y_true_bin = label_binarize(y_true, classes=range(num_classes))

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro-average ROC curve
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Macro-average
macro_roc_auc = np.mean([roc_auc[i] for i in range(num_classes)])
weighted_roc_auc = roc_auc_score(y_true_bin, y_pred_proba, average='weighted')

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = plt.cm.Set3(np.linspace(0, 1, num_classes))

for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2.5,
             label=f'{class_names[i]} (AUC = {roc_auc[i]:.3f})')

plt.plot(fpr["micro"], tpr["micro"], color='deeppink', linestyle='--', lw=2.5,
         label=f'Micro-average (AUC = {roc_auc["micro"]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.5)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontweight='bold', fontsize=12)
plt.ylabel('True Positive Rate', fontweight='bold', fontsize=12)
plt.title('📈 ROC Curves - Multi-Class Classification', fontweight='bold', fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'roc_curves.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 ROC-AUC Metrics:")
print(f"   Macro ROC-AUC:    {macro_roc_auc:.4f}")
print(f"   Weighted ROC-AUC: {weighted_roc_auc:.4f}")
print(f"   Micro ROC-AUC:    {roc_auc['micro']:.4f}")
print("\n   Per-class ROC-AUC:")
for i, cls in enumerate(class_names):
    print(f"      {cls}: {roc_auc[i]:.4f}")

## 1️⃣4️⃣ Precision-Recall Curves

In [ ]:
# Calculate Precision-Recall for each class
precision = dict()
recall = dict()
pr_auc = dict()

for i in range(num_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_pred_proba[:, i])
    pr_auc[i] = average_precision_score(y_true_bin[:, i], y_pred_proba[:, i])

# Micro-average
precision["micro"], recall["micro"], _ = precision_recall_curve(
    y_true_bin.ravel(), y_pred_proba.ravel()
)
pr_auc["micro"] = average_precision_score(y_true_bin, y_pred_proba, average="micro")

# Macro and weighted averages
macro_pr_auc = np.mean([pr_auc[i] for i in range(num_classes)])
weighted_pr_auc = average_precision_score(y_true_bin, y_pred_proba, average='weighted')

# Plot PR curves
plt.figure(figsize=(10, 8))
colors = plt.cm.Set3(np.linspace(0, 1, num_classes))

for i, color in zip(range(num_classes), colors):
    plt.plot(recall[i], precision[i], color=color, lw=2.5,
             label=f'{class_names[i]} (AP = {pr_auc[i]:.3f})')

plt.plot(recall["micro"], precision["micro"], color='deeppink', linestyle='--', lw=2.5,
         label=f'Micro-average (AP = {pr_auc["micro"]:.3f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontweight='bold', fontsize=12)
plt.ylabel('Precision', fontweight='bold', fontsize=12)
plt.title('📊 Precision-Recall Curves', fontweight='bold', fontsize=14)
plt.legend(loc="lower left", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'precision_recall_curves.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Precision-Recall Metrics:")
print(f"   Macro PR-AUC:    {macro_pr_auc:.4f}")
print(f"   Weighted PR-AUC: {weighted_pr_auc:.4f}")
print(f"   Micro PR-AUC:    {pr_auc['micro']:.4f}")
print("\n   Per-class Average Precision:")
for i, cls in enumerate(class_names):
    print(f"      {cls}: {pr_auc[i]:.4f}")

## 1️⃣5️⃣ Per-Class Performance Metrics

In [ ]:
# Create comprehensive per-class metrics table
metrics_data = []
for i, cls in enumerate(class_names):
    metrics_data.append({
        'Class': cls,
        'Precision': report[cls]['precision'],
        'Recall': report[cls]['recall'],
        'F1-Score': report[cls]['f1-score'],
        'Support': int(report[cls]['support']),
        'ROC-AUC': roc_auc[i],
        'PR-AUC': pr_auc[i]
    })

metrics_df = pd.DataFrame(metrics_data)

# Plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Per-Class Performance Metrics', fontsize=16, fontweight='bold')

# Precision, Recall, F1
x = np.arange(len(class_names))
width = 0.25

axes[0, 0].bar(x - width, metrics_df['Precision'], width, label='Precision', alpha=0.8, color='#3498db')
axes[0, 0].bar(x, metrics_df['Recall'], width, label='Recall', alpha=0.8, color='#2ecc71')
axes[0, 0].bar(x + width, metrics_df['F1-Score'], width, label='F1-Score', alpha=0.8, color='#e74c3c')
axes[0, 0].set_xlabel('Class', fontweight='bold')
axes[0, 0].set_ylabel('Score', fontweight='bold')
axes[0, 0].set_title('Precision, Recall, F1-Score', fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(class_names, rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].set_ylim([0, 1.1])
axes[0, 0].grid(True, alpha=0.3, axis='y')

# ROC-AUC and PR-AUC
axes[0, 1].bar(x - width/2, metrics_df['ROC-AUC'], width, label='ROC-AUC', alpha=0.8, color='#9b59b6')
axes[0, 1].bar(x + width/2, metrics_df['PR-AUC'], width, label='PR-AUC', alpha=0.8, color='#f39c12')
axes[0, 1].set_xlabel('Class', fontweight='bold')
axes[0, 1].set_ylabel('AUC Score', fontweight='bold')
axes[0, 1].set_title('ROC-AUC vs PR-AUC', fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(class_names, rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].set_ylim([0, 1.1])
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Support distribution
axes[1, 0].bar(class_names, metrics_df['Support'], alpha=0.8, color='#1abc9c')
axes[1, 0].set_xlabel('Class', fontweight='bold')
axes[1, 0].set_ylabel('Number of Samples', fontweight='bold')
axes[1, 0].set_title('Validation Set Distribution', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(metrics_df['Support']):
    axes[1, 0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Metrics table
axes[1, 1].axis('tight')
axes[1, 1].axis('off')
table_data = [[row['Class'], f"{row['Precision']:.3f}", f"{row['Recall']:.3f}",
               f"{row['F1-Score']:.3f}", f"{row['ROC-AUC']:.3f}", f"{row['PR-AUC']:.3f}", row['Support']]
              for _, row in metrics_df.iterrows()]
table = axes[1, 1].table(cellText=table_data,
                        colLabels=['Class', 'Precision', 'Recall', 'F1', 'ROC-AUC', 'PR-AUC', 'Support'],
                        cellLoc='center', loc='center', colWidths=[0.15, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
axes[1, 1].set_title('Detailed Metrics Table', fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'per_class_metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Per-class metrics visualization saved")

## 1️⃣6️⃣ Comprehensive Metrics Summary

In [ ]:
# Create comprehensive summary visualization
fig = plt.figure(figsize=(16, 8))
fig.suptitle('🎯 Comprehensive Performance Summary', fontsize=16, fontweight='bold')

# Overall metrics bar chart
ax1 = plt.subplot(121)
overall_metrics = {
    'Accuracy': overall_accuracy,
    'Macro F1': macro_f1,
    'Weighted F1': weighted_f1,
    'MCC': mcc,
    'Macro ROC-AUC': macro_roc_auc,
    'Weighted ROC-AUC': weighted_roc_auc,
    'Macro PR-AUC': macro_pr_auc
}

colors_gradient = plt.cm.viridis(np.linspace(0.3, 0.9, len(overall_metrics)))
bars = ax1.barh(list(overall_metrics.keys()), list(overall_metrics.values()),
                color=colors_gradient, alpha=0.8)
ax1.set_xlabel('Score', fontweight='bold', fontsize=12)
ax1.set_title('Overall Performance Metrics', fontweight='bold', fontsize=12)
ax1.set_xlim([0, 1])
ax1.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for bar in bars:
    width = bar.get_width()
    ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2,
             f'{width:.3f}', ha='left', va='center', fontweight='bold', fontsize=10)

# Radar chart for per-class performance
categories = ['Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'PR-AUC']
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

ax2 = plt.subplot(122, projection='polar')
colors_radar = plt.cm.Set3(np.linspace(0, 1, num_classes))

for i, (cls, color) in enumerate(zip(class_names, colors_radar)):
    values = [
        report[cls]['precision'],
        report[cls]['recall'],
        report[cls]['f1-score'],
        roc_auc[i],
        pr_auc[i]
    ]
    values += values[:1]
    ax2.plot(angles, values, 'o-', linewidth=2.5, label=cls, color=color)
    ax2.fill(angles, values, alpha=0.2, color=color)

ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(categories, fontsize=10)
ax2.set_ylim(0, 1)
ax2.set_title('Per-Class Performance Radar', fontweight='bold', fontsize=12, pad=20)
ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
ax2.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'comprehensive_summary.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comprehensive summary visualization saved")

## 1️⃣7️⃣ Save All Metrics to JSON

In [ ]:
all_metrics = {
    'model_architecture': 'CNN-BiLSTM',
    'overall': {
        'accuracy': float(overall_accuracy),
        'macro_f1': float(macro_f1),
        'weighted_f1': float(weighted_f1),
        'matthews_corrcoef': float(mcc),
        'macro_roc_auc': float(macro_roc_auc),
        'weighted_roc_auc': float(weighted_roc_auc),
        'micro_roc_auc': float(roc_auc['micro']),
        'macro_pr_auc': float(macro_pr_auc),
        'weighted_pr_auc': float(weighted_pr_auc),
        'micro_pr_auc': float(pr_auc['micro'])
    },
    'per_class': {}
}

for i, cls in enumerate(class_names):
    all_metrics['per_class'][cls] = {
        'precision': float(report[cls]['precision']),
        'recall': float(report[cls]['recall']),
        'f1_score': float(report[cls]['f1-score']),
        'support': int(report[cls]['support']),
        'roc_auc': float(roc_auc[i]),
        'pr_auc': float(pr_auc[i])
    }

metrics_file = os.path.join(MODEL_DIR, 'all_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"✅ All metrics saved to: {metrics_file}")

## 1️⃣8️⃣ Save Model and Artifacts

In [ ]:
# Save final model
final_model_path = os.path.join(MODEL_DIR, "final_cnn_bilstm.keras")
model.save(final_model_path)

# Save label mapping
label_map = {
    'idx_to_name': idx_to_name,
    'name_to_idx': name_to_idx,
    'num_classes': num_classes
}
with open(os.path.join(MODEL_DIR, 'label_map.json'), 'w') as f:
    json.dump(label_map, f, indent=2)

print("\n" + "="*80)
print("💾 MODEL AND ARTIFACTS SAVED")
print("="*80)
print(f"✅ Best checkpoint:  {best_ckpt}")
print(f"✅ Final model:      {final_model_path}")
print(f"✅ Artifacts dir:    {MODEL_DIR}")
print("\n📁 Saved files:")
for f in sorted(os.listdir(MODEL_DIR)):
    print(f"   - {f}")
print("="*80)

## 1️⃣9️⃣ Inference Helper Class

In [ ]:
class AppleDiseasePredictor:
    """Helper class for making predictions on new images."""

    def __init__(self, model_path, label_map_path):
        self.model = tf.keras.models.load_model(model_path)
        with open(label_map_path, 'r') as f:
            label_map = json.load(f)
        self.idx_to_name = {int(k): v for k, v in label_map['idx_to_name'].items()}
        self.num_classes = label_map['num_classes']

    def preprocess_image(self, image_path):
        """Preprocess a single image for prediction."""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, IMG_SIZE)
        img = tf.keras.applications.efficientnet.preprocess_input(img * 255.0)
        img = tf.expand_dims(img, 0)
        return img

    def predict(self, image_path, top_k=None):
        """Make prediction on an image."""
        img = self.preprocess_image(image_path)
        predictions = self.model.predict(img, verbose=0)[0]

        if top_k is None:
            top_k = self.num_classes

        top_indices = np.argsort(predictions)[::-1][:top_k]
        results = []
        for idx in top_indices:
            results.append({
                'class': self.idx_to_name[idx],
                'confidence': float(predictions[idx])
            })
        return results

print("✅ AppleDiseasePredictor class defined")
print("\n💡 Example usage:")
print(f"   predictor = AppleDiseasePredictor('{best_ckpt}', '{os.path.join(MODEL_DIR, 'label_map.json')}')")
print("   results = predictor.predict('path/to/image.jpg')")
print("   print(results)")

## 2️⃣0️⃣ Test Inference on Sample Images

In [ ]:
# Initialize predictor
predictor = AppleDiseasePredictor(best_ckpt, os.path.join(MODEL_DIR, 'label_map.json'))

# Get random validation samples
sample_indices = np.random.choice(len(val_recs), min(9, len(val_recs)), replace=False)
sample_recs = [val_recs[i] for i in sample_indices]

# Create visualization
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('🔮 Sample Predictions on Validation Set', fontsize=16, fontweight='bold')
axes = axes.ravel()

for idx, rec in enumerate(sample_recs):
    # Load and display image
    img = plt.imread(rec['path'])
    axes[idx].imshow(img)
    axes[idx].axis('off')

    # Get prediction
    results = predictor.predict(rec['path'], top_k=3)

    # Create title
    true_label = rec['label_name']
    pred_label = results[0]['class']
    confidence = results[0]['confidence']

    # Color code: green if correct, red if wrong
    color = 'green' if true_label == pred_label else 'red'

    # Build title with top predictions
    title = f"True: {true_label}\n"
    title += f"Pred: {pred_label} ({confidence:.1%})\n"
    if len(results) > 1:
        title += f"2nd: {results[1]['class']} ({results[1]['confidence']:.1%})"

    axes[idx].set_title(title, color=color, fontweight='bold', fontsize=9, pad=5)

    # Add border
    for spine in axes[idx].spines.values():
        spine.set_edgecolor(color)
        spine.set_linewidth(3)

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'sample_predictions.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Sample predictions saved")

# Calculate accuracy on samples
correct = sum(1 for rec in sample_recs if predictor.predict(rec['path'])[0]['class'] == rec['label_name'])
print(f"\n📊 Sample accuracy: {correct}/{len(sample_recs)} ({correct/len(sample_recs)*100:.1f}%)")

## 2️⃣1️⃣ Final Summary

In [ ]:
print("\n" + "="*80)
print("🎉 TRAINING AND EVALUATION COMPLETE!")
print("="*80)

print("\n📊 KEY RESULTS:")
print(f"   Overall Accuracy:       {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
print(f"   Macro F1-Score:         {macro_f1:.4f}")
print(f"   Weighted F1-Score:      {weighted_f1:.4f}")
print(f"   Matthews Corr Coef:     {mcc:.4f}")
print(f"   Macro ROC-AUC:          {macro_roc_auc:.4f}")
print(f"   Weighted ROC-AUC:       {weighted_roc_auc:.4f}")
print(f"   Macro PR-AUC:           {macro_pr_auc:.4f}")
print(f"   Weighted PR-AUC:        {weighted_pr_auc:.4f}")

print("\n📁 GENERATED ARTIFACTS:")
artifacts = [
    'training_history.png',
    'confusion_matrices.png',
    'roc_curves.png',
    'precision_recall_curves.png',
    'per_class_metrics.png',
    'comprehensive_summary.png',
    'sample_predictions.png',
    'classification_report.csv',
    'all_metrics.json',
    'best_cnn_bilstm.keras',
    'final_cnn_bilstm.keras',
    'label_map.json'
]

for artifact in artifacts:
    if os.path.exists(os.path.join(MODEL_DIR, artifact)):
        print(f"   ✅ {artifact}")
    else:
        print(f"   ⚠️  {artifact} (not found)")

print(f"\n📂 All artifacts saved in: {MODEL_DIR}")

print("\n" + "="*80)
print("🚀 READY FOR DEPLOYMENT!")
print("="*80)

print("\n💡 Next Steps:")
print("   1. Review the comprehensive visualizations and metrics")
print("   2. Test the model on new images using AppleDiseasePredictor")
print("   3. Analyze per-class performance for potential improvements")
print("   4. Deploy the model for production use")
print("   5. Monitor model performance on real-world data")
print("\n" + "="*80)